In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import gen_fbm_nn_model as fbm_nn

In [5]:
def load_nn_model(window_size,n_samples=10000,n_epochs=100):

    try:
        model = tf.keras.models.load_model("model3dense_n{}.h5".format(window_size))
    except OSError:
        fbm_nn.__main__(n_samples,window_size,n_epochs)
        model = tf.keras.models.load_model("model3dense_n{}.h5".format(window_size))
    return model

def get_model_metrics(window_size,n_samples=10000,n_epochs=100):
    
    model = load_nn_model(window_size, n_samples, n_epochs)
    
    loss, mean_abs_error, mean_sqr_error = np.loadtxt('metrics_'+"model3dense_n"+str(window_size), comments='#')
    
    return loss, mean_abs_error, mean_sqr_error

In [ ]:
window_sizes = [12,13,14,15,16,17,18,19,20,21,22,23,24,25]
loss_values = []
mean_abs_errors = []
mean_sqr_errors = []

for w in window_sizes:
    print('Window size: ', w)
    loss, mae, mse = get_model_metrics(w)
    loss_values.append(loss)
    mean_abs_errors.append(mae)
    mean_sqr_errors.append(mse)

Window size:  12
Window size:  13
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 13)                182       
                                                                 
 dense_9 (Dense)             (None, 12)                168       
                                                                 
 dense_10 (Dense)            (None, 11)                143       
                                                                 
 dense_11 (Dense)            (None, 1)                 12        
                                                                 
Total params: 505
Trainable params: 505
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
125/125 [==============================] - 2s 8ms/step - loss: 0.3124 - mean_absolute_error: 0.3124 - mean_squared_error: 0.1463 - val_loss: 0

125/125 [==============================] - 1s 11ms/step - loss: 0.1230 - mean_absolute_error: 0.1230 - mean_squared_error: 0.0260 - val_loss: 0.1347 - val_mean_absolute_error: 0.1347 - val_mean_squared_error: 0.0305
Epoch 33/100
125/125 [==============================] - 1s 10ms/step - loss: 0.1228 - mean_absolute_error: 0.1228 - mean_squared_error: 0.0260 - val_loss: 0.1352 - val_mean_absolute_error: 0.1352 - val_mean_squared_error: 0.0307
Epoch 34/100
125/125 [==============================] - 1s 11ms/step - loss: 0.1226 - mean_absolute_error: 0.1226 - mean_squared_error: 0.0258 - val_loss: 0.1355 - val_mean_absolute_error: 0.1355 - val_mean_squared_error: 0.0308
Epoch 35/100
125/125 [==============================] - 1s 11ms/step - loss: 0.1219 - mean_absolute_error: 0.1219 - mean_squared_error: 0.0257 - val_loss: 0.1352 - val_mean_absolute_error: 0.1352 - val_mean_squared_error: 0.0307
Epoch 36/100
125/125 [==============================] - 1s 12ms/step - loss: 0.1229 - mean_absolu

Epoch 68/100
125/125 [==============================] - 1s 10ms/step - loss: 0.1184 - mean_absolute_error: 0.1184 - mean_squared_error: 0.0243 - val_loss: 0.1331 - val_mean_absolute_error: 0.1331 - val_mean_squared_error: 0.0302
Epoch 69/100
125/125 [==============================] - 1s 8ms/step - loss: 0.1182 - mean_absolute_error: 0.1182 - mean_squared_error: 0.0243 - val_loss: 0.1332 - val_mean_absolute_error: 0.1332 - val_mean_squared_error: 0.0303
Epoch 70/100
125/125 [==============================] - 1s 10ms/step - loss: 0.1182 - mean_absolute_error: 0.1182 - mean_squared_error: 0.0242 - val_loss: 0.1332 - val_mean_absolute_error: 0.1332 - val_mean_squared_error: 0.0303
Epoch 71/100
125/125 [==============================] - 1s 10ms/step - loss: 0.1182 - mean_absolute_error: 0.1182 - mean_squared_error: 0.0243 - val_loss: 0.1334 - val_mean_absolute_error: 0.1334 - val_mean_squared_error: 0.0302
Epoch 72/100
125/125 [==============================] - 1s 8ms/step - loss: 0.1191 - 

                                                                 
 dense_15 (Dense)            (None, 1)                 13        
                                                                 
Total params: 586
Trainable params: 586
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
125/125 [==============================] - 1s 6ms/step - loss: 0.2916 - mean_absolute_error: 0.2916 - mean_squared_error: 0.1268 - val_loss: 0.1926 - val_mean_absolute_error: 0.1926 - val_mean_squared_error: 0.0540
Epoch 2/100
125/125 [==============================] - 0s 4ms/step - loss: 0.1641 - mean_absolute_error: 0.1641 - mean_squared_error: 0.0431 - val_loss: 0.1523 - val_mean_absolute_error: 0.1523 - val_mean_squared_error: 0.0376
Epoch 3/100
125/125 [==============================] - 1s 9ms/step - loss: 0.1463 - mean_absolute_error: 0.1463 - mean_squared_error: 0.0358 - val_loss: 0.1456 - val_mean_absolute_error: 0.1456 - val_mean_squared_error

In [ ]:
plt.plot(window_sizes, mean_abs_errors)
#plt.plot(window_sizes, mean_sqr_errors)